In [ ]:
# pip install tensorflow

In [ ]:
import keras_tuner as kt
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras_tuner as kt
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error


In [ ]:

# 定义模型构建函数
def build_model(hp):
    try:
        # 调整隐藏层数量
        n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
        
        # 调整每层神经元数量
        n_neurons = hp.Int("n_neurons", min_value=16, max_value=256)
        
        # 调整cell数量
        n_cells = hp.Int("n_cells", min_value=1, max_value=10)

        # 调整学习率
        learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-1, sampling="log")

        # 选择优化器
        optimizer_choice = hp.Choice("optimizer", values=["adam", "sgd", "adamax", "rmsprop", "adadelta", "adagrad", "nadam"])
        
        # 选择批量大小
        batch_size = hp.Int("batch_size", min_value=16, max_value=128, step=16)

        # 根据选择的优化器设定优化器
        if optimizer_choice == "adam":
            optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        elif optimizer_choice == "sgd":
            optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
        elif optimizer_choice == "adamax":
            optimizer = tf.keras.optimizers.Adamax(learning_rate=learning_rate)
        elif optimizer_choice == "rmsprop":
            optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
        elif optimizer_choice == "adadelta":
            optimizer = tf.keras.optimizers.Adadelta(learning_rate=learning_rate)
        elif optimizer_choice == "adagrad":
            optimizer = tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
        elif optimizer_choice == "nadam":
            optimizer = tf.keras.optimizers.Nadam(learning_rate=learning_rate)

        # 创建模型
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Flatten())

        # 添加隐藏层
        for _ in range(n_hidden):
            for _ in range(n_cells):
                model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))

        # 添加输出层
        model.add(tf.keras.layers.Dense(10, activation="softmax"))

        # 编译模型
        model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
        
        return model

    except Exception as e:
        print(f"构建模型时出错: {e}")
        return None


In [ ]:
# 读取数据集
data = pd.read_csv('data.csv')
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values